#### Extraer Consulta de Datos sobre Financiamientos

El presente código tiene como objetivo crear una visualización final de la consulta de datos sobre operaciones financieras 

In [2]:
# Instalar librería kaleido para guardar imagenes
!pip install --force-reinstall kaleido==0.1.0post1

  Using cached kaleido-0.1.0.post1-py2.py3-none-win_amd64.whl.metadata (15 kB)
Using cached kaleido-0.1.0.post1-py2.py3-none-win_amd64.whl (56.0 MB)
  Attempting uninstall: kaleido
    Found existing installation: kaleido 0.1.0.post1
    Uninstalling kaleido-0.1.0.post1:
      Successfully uninstalled kaleido-0.1.0.post1


In [3]:
# Importar librerías necesarias
import json
import matplotlib.pyplot as plt
from matplotlib import rcParams
import psycopg2
import os
import re
import csv
import platform
import subprocess
import pandas as pd
import plotly.graph_objects as go

In [16]:
# Establecer la ruta del directorio donde trabajaremos
ruta_proyecto = r'C:\Users\quetz\Documents'  #Agregar la ruta que necesitamos
os.chdir(ruta_proyecto)  # Ejecutar nuestra ruta 

# Verificar que se ha cambiado la ruta
print("Ruta actual 📂:", os.getcwd())


Ruta actual 📂: C:\Users\quetz\Documents


In [17]:
# Parámetros de conexión / accesos
DB = {
    'host': '*******',
    'database': 'biprod',
    'user': 'alejandra_gonzalez',
    'password': 'AGonzalez123.$'
}

try:
    # Conexión a la base de datos
    conn = psycopg2.connect(**DB)
    cur = conn.cursor()

    # Verificación de la conexión
    print("Conexión exitosa a la base de datos 🥳✅")

    # Consulta postgress
    query = """
    SELECT 
        l.usuarioCapturo,
        l.nombreEmpleado,
        l.puestoEmpleado,
        to_char(l.fechaSolicitud, 'DD/MM/YYYY') AS fechaSolicitud,
        COALESCE(NULLIF(l.region, 'NACIONAL'), 'CIUDAD DE MEXICO') AS region,
        COALESCE(NULLIF(l.estado, 'CALL CENTER'), 'CORPORATIVO') AS estado,
        COALESCE(NULLIF(l.sucursal, 'BROKER MEXI'), 'CORPORATIVO') AS sucursal,
        COALESCE(
            (SELECT UPPER(r.descripcion)
             FROM acle.erp_catsucursales s
             JOIN acle.erp_catregion r ON s.idregion = r.idregion
             WHERE UPPER(s.nombresucursal) = UPPER(
                COALESCE(p.sucursal, pp.sucursal, ppp.sucursal, 'CORPORATIVO'))
             AND fechainactividad IS NULL 
             LIMIT 1), 
            'CORPORATIVO'
        ) AS regionAsesor,
        COALESCE(p.estado, pp.estado, ppp.estado, 'CIUDAD DE MEXICO') AS estadoAsesor,
        COALESCE(p.sucursal, pp.sucursal, ppp.sucursal, 'CORPORATIVO') AS sucursalAsesor,
        l.numeroSolicitud,
        l.numeroCliente,
        l.nombreCliente,
        l.folioSolicitud,
        dh.description AS dependenciaHomologada,
        cp.lineaNegocio,
        cp.descripcionCOE,
        l.tipoDispersion,
        l.tipoPromocion,
        l.pagos,
        split_part(l.nombreGerente, ',', 1) AS cvegerente,
        TRIM(split_part(l.nombreGerente, ',', 2)) AS nombreGerente,
        split_part(l.nombreSupervisor, ',', 1) AS cvesupervisor,
        TRIM(split_part(l.nombreSupervisor, ',', 2)) AS nombreSupervisor,
        split_part(l.nombreAsesor, ',', 1) AS cveasesor,
        TRIM(split_part(l.nombreAsesor, ',', 2)) AS nombreAsesor,
        to_char(b.fechaApertura, 'DD/MM/YYYY') AS fechaApertura,
        (l.montoCobrado + l.montoACobrar) AS montoColocacion,
        l.fechaCobro,
        l.numeroCredito,
        l.estatusC,
        l.estatusD,
        l.estatus,
        l.fechaHoraInicioCaptura,
        l.fechaHoraFinCaptura,
        l.fechaEstatus,
        l.horaEstatus,
        l.horasOperativas,
        l.totalDevolucionesZell,
        l.totalDevolucionesAnalistas,
        l.lineaCredito,
        l.seguro,
        l.banco,
        l.subtipoPromocion,
        l.producto,
        b.dap AS cveDap,
        (SELECT od.etiqueta
         FROM dwhprod.dwh_bidatocomun dc
         JOIN dwhprod.dwh_catopciondato od ON dc.iddato = od.iddato AND dc.valordato = od.valor
         WHERE dc.numerocredito = l.numeroSolicitud
           AND dc.idreferencia = 9
           AND dc.iddato = 95
         LIMIT 1) AS tipoNombramiento,
        (SELECT od.etiqueta
         FROM dwhprod.dwh_bidatocomun dc
         JOIN dwhprod.dwh_catopciondato od ON dc.iddato = od.iddato AND dc.valordato = od.valor
         WHERE dc.numerocredito = l.numeroSolicitud
           AND dc.idreferencia = 9
           AND dc.iddato = 96
         LIMIT 1) AS tipoEmpleado,
        l.genero,
        p.puestoAsesor,
        CASE 
            WHEN t.tasaGlobal IS NOT NULL 
            THEN CONCAT_WS('', TO_CHAR(CAST(t.tasaGlobal AS NUMERIC), '99D0999'), '%') 
            ELSE '' 
        END AS valorTasaGlobal,
        l.dependencia
    FROM 
        dwhprod.dwh_vwTiemposOriginacion2 l
    LEFT JOIN dwhprod.dwh_bicredito b ON l.numeroSolicitud = b.numeroSolicitud
    LEFT JOIN dwhprod.dwh_biplantillaempleados p ON split_part(l.nombreAsesor, ',', 1) = p.idEmpleado
    LEFT JOIN dwhprod.dwh_biplantillaempleados pp ON split_part(l.nombreSupervisor, ',', 1) = pp.idEmpleado
    LEFT JOIN dwhprod.dwh_biplantillaempleados ppp ON split_part(l.nombreGerente, ',', 1) = ppp.idEmpleado
    LEFT JOIN dwhprod.dwh_catDependencias dd ON b.idDependencia = dd.idDependencia
    LEFT JOIN dataprod.catafiliatehm dhm ON UPPER(dd.cveDependencia) = UPPER(dhm.afiliateid)
    LEFT JOIN dataprod.catgovermentdependency dh ON dhm.govermentdependencyid = dh.govermentdependencyid
    LEFT JOIN dwhprod.dwh_catproductos cp ON b.idProducto = cp.idProducto
    LEFT JOIN dwhprod.tblcreddictaminacion t ON b.numeroCredito = t.idCredito;
    """
    
    # Ejecutar la consulta
    cur.execute(query)

    # Recuperar los resultados
    rows = cur.fetchall()
    column_names = [desc[0] for desc in cur.description]

    # Guardar resultados en un archivo CSV
    with open('consulta_resultados.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(column_names)  # Escribir encabezados
        writer.writerows(rows)         # Escribir datos

    print("Consulta ejecutada y resultados guardados en 'consulta_resultados.csv'")

except psycopg2.Error as e:
    print(f"Error en la conexión o consulta: {e}")

finally:
    if cur:
        cur.close()
    if conn:
        conn.close()
        

Conexión exitosa a la base de datos 🥳✅
Consulta ejecutada y resultados guardados en 'consulta_resultados.csv'


In [18]:
archivo_salida = 'resultados_consulta.csv'
ruta = os.path.abspath(archivo_salida)
print(f"El archivo se guardó en la ruta 📂✅: {ruta}")


El archivo se guardó en la ruta 📂✅: C:\Users\quetz\Documents\resultados_consulta.csv


In [19]:
# Ruta del archivop csv
csv_path = 'consulta_resultados.csv'

In [20]:
import pandas as pd

# Leer el archivo CSV
df = pd.read_csv(csv_path, encoding='utf-8', on_bad_lines='skip')

# Agrupar por estatus y calcular el número de solicitudes y el monto total
resultado = df.groupby('estatus').agg(
    num_solicitudes=('numerosolicitud', 'count'),
    monto_total=('montocolocacion', 'sum')
).reset_index()

# Calcular la suma de los totales
total_solicitudes = resultado['num_solicitudes'].sum()
total_monto = resultado['monto_total'].sum()

# Crear un nuevo DataFrame con los totales
totales = pd.DataFrame({
    'estatus': ['Total'],
    'num_solicitudes': [total_solicitudes],
    'monto_total': [total_monto]
})

# Concatenar los totales con el resultado original
resultado = pd.concat([resultado, totales], ignore_index=True)
resultado


,estatus,num_solicitudes,monto_total
0,Analisis CF,24,765364.82
1,Aperturada,6,150869.48
2,Aperturada y Cobrada,701,16849133.37
3,Autorizada,2,40000.00
4,Cancelada,89,2222482.95
5,Capturada,27,611435.68
6,Colocacion Portal,9,146386.21
7,Devolucion CF,21,887970.53
8,Mesa de Control,6,157958.49
9,Pendiente por Sistema,2,126960.00


In [37]:
import pandas as pd

# Leer el archivo CSV
df = pd.read_csv(csv_path, encoding='utf-8', on_bad_lines='skip')

# Agrupar por 'estatus' y calcular el número de solicitudes y la suma del monto
resultado = df.groupby('estatus').agg(
    num_solicitudes=('numerosolicitud', 'count'),
    monto_total=('montocolocacion', 'sum')
).reset_index()

# Calcular la suma total de solicitudes y montos
total_fila = pd.DataFrame({
    'estatus': ['Total'],
    'num_solicitudes': [resultado['num_solicitudes'].sum()],
    'monto_total': [resultado['monto_total'].sum()]
})

# Usar concat para agregar la fila de la suma total al resultado
resultado = pd.concat([resultado, total_fila], ignore_index=True)

# Definir las categorías que deben incluirse en 'TUBERIA'
e_tuberia = [
    'Analisis CF', 'Colocacion  Portal', 'Mesa de Control', 
    'Pendiente CR', 'Pendiente por Sistema', 'Por Autorizar Refiel', 'Por Formalizar',
    'Sin contacto telefónico', 'Validación con ADC', 'ValidacionTelefonica', 
    'Validata', 'VoBo de Calidad', 'Por Autorizar', 
    'Pendiente por cambio de anexo o política'
]

# Filtrar las categorías de 'TUBERIA' y calcular los totales
tuberia_df = resultado[resultado['estatus'].isin(e_tuberia)]
tuberia_totales = {
    'estatus': 'TUBERIA',
    'num_solicitudes': tuberia_df['num_solicitudes'].sum(),
    'monto_total': tuberia_df['monto_total'].sum()
}

# Convertir 'TUBERIA' a DataFrame
tuberia_fila = pd.DataFrame([tuberia_totales])

# Agregar la fila 'TUBERIA' después de la fila 'Total'
tuberia_p = resultado[resultado['estatus'] == 'Total'].index[0]
resultado = pd.concat([resultado.iloc[:tuberia_p+1], tuberia_fila, resultado.iloc[tuberia_p+1:]], ignore_index=True)

# Definir la fila de 'Mes Pasado Comprobado'
mes_pasado_comprobado = {
    'estatus': 'Mes Pasado Comprobado',
    'num_solicitudes': 80,  # Agregado manualmente
    'monto_total': 2762042  # Agregado manualmente
}

# Agregar la fila 'Mes Pasado Comprobado' al inicio del df
resultado = pd.concat([pd.DataFrame([mes_pasado_comprobado]), resultado], ignore_index=True)

# Agregar 'Salvo Buen Cobro' a la columna estatus después de 'TUBERIA'
if 'Salvo Buen Cobro' in resultado['estatus'].values:
    salvo_buen_cobro = resultado.loc[resultado['estatus'] == 'Salvo Buen Cobro'].iloc[0]
else:
    salvo_buen_cobro = {
        'estatus': 'Salvo Buen Cobro',
        'num_solicitudes': 0,
        'monto_total': 0 }

# Crear DataFrame de 'Salvo Buen Cobro' y agregarlo después de la fila 'TUBERIA'
salvo_bc = resultado[resultado['estatus'] == 'TUBERIA'].index[0]
salvo_buen_cobro_df = pd.DataFrame([salvo_buen_cobro])
resultado = pd.concat([resultado.iloc[:salvo_bc+1], salvo_buen_cobro_df, resultado.iloc[salvo_bc+1:]], ignore_index=True)

# Sumar los valores de 'Aperturada y Cobrada' y 'Mes Pasado Comprobado'
aperturada_y_cobrada = resultado[resultado['estatus'].isin(['Aperturada y Cobrada', 'Mes Pasado Comprobado'])].agg({
    'num_solicitudes': 'sum',
    'monto_total': 'sum'
}).to_dict()

# Crear la fila de 'Aperturada y Cobrada'
aperturada_y_cobrada_fila = {
    'estatus': 'Aperturada y Cobrada.',
    'num_solicitudes': aperturada_y_cobrada['num_solicitudes'],
    'monto_total': aperturada_y_cobrada['monto_total']
}

# Convertir la fila a un DataFrame
aperturada_y_cobrada_df = pd.DataFrame([aperturada_y_cobrada_fila])

# Agregar la fila 'Aperturada y Cobrada' después de la fila 'Salvo Buen Cobro'
salvo_buen_cobro_p = resultado[resultado['estatus'] == 'Salvo Buen Cobro'].index[0]
resultado = pd.concat([resultado.iloc[:salvo_buen_cobro_p+1], aperturada_y_cobrada_df, resultado.iloc[salvo_buen_cobro_p+1:]], ignore_index=True)

# Calcular el porcentaje total de solicitudes
resultado['porcentaje'] = resultado.apply(
    lambda row: (row['monto_total'] / resultado.loc[resultado['estatus'] == 'Total', 'monto_total'].values[0]) * 100
    if row['estatus'] not in ['Total', 'TUBERIA', 'Salvo Buen Cobro', 'Aperturada y Cobrada.', 'Mes Pasado Comprobado'] else None,
    axis=1
)

# Porcentaje 'TUBERIA'
tuberia_solicitudes = resultado.loc[resultado['estatus'] == 'TUBERIA', 'num_solicitudes'].values[0]
resultado.loc[resultado['estatus'] == 'TUBERIA', 'porcentaje'] = round((tuberia_solicitudes / resultado.loc[resultado['estatus'] == 'Total', 'num_solicitudes'].values[0]) * 100, 2)

# Porcentaje 'Salvo Buen Cobro'
salvo_buen_cobro_solicitudes = resultado.loc[resultado['estatus'] == 'Salvo Buen Cobro', 'num_solicitudes'].values[0]
resultado.loc[resultado['estatus'] == 'Salvo Buen Cobro', 'porcentaje'] = round((tuberia_solicitudes / salvo_buen_cobro_solicitudes) * 100, 2) if salvo_buen_cobro_solicitudes > 0 else 0

# Porcentaje 'Total'
estatus_excluidos = ['TUBERIA', 'Salvo Buen Cobro', 'Aperturada y Cobrada.', 'Mes Pasado Comprobado', 'Pre Capturada']
total_ptc = resultado.loc[~resultado['estatus'].isin(estatus_excluidos), 'porcentaje'].sum()
resultado.loc[resultado['estatus'] == 'Total', 'porcentaje'] = round(total_ptc, 2)

# Porcentajes a 2 decimales
resultado['porcentaje'] = resultado['porcentaje'].round(2)

# Convertir 'num_solicitudes' a enteros
resultado['num_solicitudes'] = resultado['num_solicitudes'].astype(int)
resultado

,estatus,num_solicitudes,monto_total,porcentaje
0,Mes Pasado Comprobado,80,2762042.00,NaN
1,Analisis CF,24,765364.82,2.41
2,Aperturada,6,150869.48,0.48
3,Aperturada y Cobrada,701,16849133.37,53.12
4,Autorizada,2,40000.00,0.13
5,Cancelada,89,2222482.95,7.01
6,Capturada,27,611435.68,1.93
7,Colocacion Portal,9,146386.21,0.46
8,Devolucion CF,21,887970.53,2.80
9,Mesa de Control,6,157958.49,0.50


In [38]:
import pandas as pd
import plotly.graph_objects as go

# Función para formatear los montos con el signo de peso y dos decimales
def formatear_montos(df, column):
    if column == 'porcentaje':
        df[column] = df[column].apply(lambda x: f"{x:.2f}%" if pd.notnull(x) else '')
    else:
        df[column] = df[column].apply(lambda x: f"${x:,.2f}" if pd.notnull(x) else '')
    return df

# Aplicar el formato de montos en las columnas que corresponden
resultado = formatear_montos(resultado, 'monto_total')
resultado = formatear_montos(resultado, 'porcentaje')

# Definir colores y formatos específicos
header_color = '#44b3e1'
default_bg_color = 'white'
mes_pasado_color = '#E97132'
tuberia_total_color = '#8eaadc'
highlight_colors = {
    'Aperturada y Cobrada': '#ffc000',
    'Aperturada y Cobrada.': '#002060',
    'Devolucion CF': '#fbe2d5',
    'Devolucion CR': '#fbe2d5',
    'Devolucion Mes Anterior': '#94dcf8',
    'Por Formalizar': '#8ed973',
    'Rechazo CF': '#fff2ce',
    'Rechazo CR': '#caedfb',
    'Total': tuberia_total_color,
    'TUBERIA': tuberia_total_color,
    'Salvo Buen Cobro': '#dae1f3',
    'Mes Pasado Comprobado': default_bg_color
}
highlight_text_colors = {
    'Aperturada y Cobrada.': 'white',
    'Mes Pasado Comprobado': mes_pasado_color,
    'Salvo Buen Cobro': 'red'
}

# Crear la figura de Plotly
fig = go.Figure()

# Agregar la fila "Mes Pasado Comprobado" arriba
fig.add_trace(go.Table(
    header=dict(
        values=['', '', '', ''],
        fill_color=default_bg_color,
        line_color=default_bg_color,
        font=dict(color=mes_pasado_color, family='sans-serif', size=14),
        align='left'
    ),
    cells=dict(
        values=[['Mes Pasado Comprobado'], ['50'], ['100000'], ['']],
        fill_color=[[default_bg_color]],
        font=dict(color=mes_pasado_color, family='sans-serif', size=14),
        align='left',
        height=30  # Ajuste para centrar verticalmente
    )
))

# Ajustar las celdas de la tabla principal para que las filas específicas no muestren NaN ni 0 en "porcentaje"
porcentaje_values = [
    ["" if (estatus in ['Mes Pasado Comprobado', 'Aperturada y Cobrada.'] and pd.isnull(val)) else val 
     for estatus, val in zip(resultado['estatus'], resultado['porcentaje'])]
]

# Agregar la tabla principal
fig.add_trace(go.Table(
    header=dict(
        values=[f'<b>{col}</b>' for col in resultado.columns],
        fill_color=header_color,
        font=dict(color='white', family='sans-serif'),
        align='center'
    ),
    cells=dict(
        values=[
            resultado[col] if col != 'porcentaje' else porcentaje_values[0] 
            for col in resultado.columns
        ],
        fill_color=[
            [highlight_colors.get(status, default_bg_color) for status in resultado['estatus']]
        ],
        font=dict(color=[
            [highlight_text_colors.get(status, 'black') for status in resultado['estatus']]
        ]),
        align=[
            'left' if resultado.columns[i] == 'estatus' and any(resultado['estatus'].str.contains(r'Aperturada y Cobrada\.')) else 'right' if resultado.columns[i] in ['monto_total', 'porcentaje_monto'] else 'center'
            for i in range(len(resultado.columns))
        ],
        height=30,  # Ajuste para centrar verticalmente
        line_color='rgba(0,0,0,0)'  # Eliminar bordes blancos
    )
))

# Ajustar el diseño con título
fig.update_layout(
    width=750,
    height=990,
    title={
        'text': "<b>Estatus, No. Solicitudes y Monto de Financiamientos</b>",
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    }
)

fig.show()

In [39]:
# Definir la ruta para guardar la imagen
ruta_imagen = r'C:\Users\quetz\Documents\financiamientos.jpg'

# Guardar la figura como imagen en formato JPG
fig.write_image(ruta_imagen)

# Imprimir mensaje de éxito con la ruta
print(f"La imagen se ha guardado correctamente en la ruta 📂✅: {ruta_imagen}")


La imagen se ha guardado correctamente en la ruta 📂✅: C:\Users\quetz\Documents\financiamientos.jpg
